In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


sys.path.append('../../src/')
from bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")

import harp
import harp.processing as hp


In [36]:
HarpBehavior = harp.Device("Behavior")
AnalogInput = harp.Device("AnalogInput")

## Paths to the data:
root = Path(r'Z:\scratch\vr-foraging\scratchpad\pid_benchmarks\20230921')
rig = "VR2"
fname_formatter = lambda reg : f"reg__{reg}.bin"
# Harp.Behavior data:
harp_behavior_root = root / rig / "Behavior"

# Harp.AnalogInput data:
harp_analog_root = root / rig / "Analog"


In [37]:
##Load digital output data
odor_valve_onset = HarpBehavior.file_to_dataframe(harp_behavior_root / fname_formatter(34))
odor_valve_offset = HarpBehavior.file_to_dataframe(harp_behavior_root / fname_formatter(35))
digital_outputs = (HarpBehavior.module.DigitalOutputs)

odor1_state = hp.distinct_until_changed_state(odor_valve_onset, odor_valve_offset, digital_outputs.SupplyPort1)
odor2_state = hp.distinct_until_changed_state(odor_valve_onset, odor_valve_offset, digital_outputs.SupplyPort2)



In [4]:
pid_data = harp.read_harp_bin(harp_analog_root / fname_formatter(33))
